In [2]:
!pip install tensorflow_ranking
!pip install tf2onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.4/150.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 89.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 90.8 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flatbuffers-23.5.26


In [3]:
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Reshape, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.initializers import GlorotUniform
import tensorflow as tf
import tensorflow_ranking as tfr
import tf2onnx

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
output_path = 'drive/MyDrive/CURIS/VeriX/networks/'

In [6]:
SEED = 137

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

11490434/11490434 [==============================] - 0s 0us/step


In [8]:
def eval_and_save(model):
  score = model.evaluate(x_test, y_test, verbose=0)
  print("Test loss:", score[0])
  print("Test accuracy:", score[1])
  model.summary()
  model.save(output_path + model.name + '.h5')
  model_proto, _ = tf2onnx.convert.from_keras(model, output_path=output_path + model.name + '.onnx')

10x2 fully connected

In [ ]:
model = Sequential(name='mnist-10x2')
model.add(Flatten(name='input'))
model.add(Dense(10, name='dense_1', activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(10, name='dense_2', activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(10, name='logit', kernel_initializer=GlorotUniform(seed=SEED)))
# model.add(Activation('softmax', name='output'))

In [ ]:
model.compile(# loss='categorical_crossentropy',
              # loss=tf.keras.losses.CategoricalCrossentropy(),
              loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 8s 5ms/step - loss: 0.8530 - accuracy: 0.7512 - val_loss: 0.4389 - val_accuracy: 0.8728
Epoch 2/20
469/469 [==============================] - 2s 5ms/step - loss: 0.3871 - accuracy: 0.8895 - val_loss: 0.3422 - val_accuracy: 0.9051
Epoch 3/20
469/469 [==============================] - 3s 5ms/step - loss: 0.3250 - accuracy: 0.9088 - val_loss: 0.3001 - val_accuracy: 0.9156
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.2881 - accuracy: 0.9202 - val_loss: 0.2747 - val_accuracy: 0.9237
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 0.2638 - accuracy: 0.9266 - val_loss: 0.2573 - val_accuracy: 0.9300
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.2485 - accuracy: 0.9307 - val_loss: 0.2458 - val_accuracy: 0.9323
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.2366 - accuracy: 0.9337 - val_loss: 0.2341 - val_accuracy: 0.9343
Epoch 

In [ ]:
eval_and_save(model)

Test loss: 0.20926763117313385
Test accuracy: 0.9408000111579895
Model: "mnist-10x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                7850      
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 logit (Dense)               (None, 10)                110       
                                                                 
Total params: 8,070
Trainable params: 8,070
Non-trainable params: 0
_________________________________________________________________


30x2 fully connected

In [ ]:
model = Sequential(name='mnist-30x2')
model.add(Flatten(name='input'))
model.add(Dense(30, name='dense_1', activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(30, name='dense_2', activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(10, name='logit', kernel_initializer=GlorotUniform(seed=SEED)))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 5s 7ms/step - loss: 0.5110 - accuracy: 0.8530 - val_loss: 0.2372 - val_accuracy: 0.9307
Epoch 2/20
469/469 [==============================] - 3s 6ms/step - loss: 0.2137 - accuracy: 0.9391 - val_loss: 0.1797 - val_accuracy: 0.9471
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1709 - accuracy: 0.9510 - val_loss: 0.1563 - val_accuracy: 0.9552
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1460 - accuracy: 0.9577 - val_loss: 0.1466 - val_accuracy: 0.9576
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1289 - accuracy: 0.9618 - val_loss: 0.1403 - val_accuracy: 0.9593
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1157 - accuracy: 0.9655 - val_loss: 0.1277 - val_accuracy: 0.9637
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1057 - accuracy: 0.9695 - val_loss: 0.1239 - val_accuracy: 0.9659
Epoch 

In [ ]:
eval_and_save(model)

Test loss: 0.11460776627063751
Test accuracy: 0.9699000120162964
Model: "mnist-30x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 30)                23550     
                                                                 
 dense_2 (Dense)             (None, 30)                930       
                                                                 
 logit (Dense)               (None, 10)                310       
                                                                 
Total params: 24,790
Trainable params: 24,790
Non-trainable params: 0
_________________________________________________________________


100x2 fully connected

In [ ]:
model = Sequential(name='mnist-100x2')
model.add(Flatten(name='input'))
model.add(Dense(100, name='dense_1', activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(100, name='dense_2', activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(10, name='logit', kernel_initializer=GlorotUniform(seed=SEED)))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 4s 6ms/step - loss: 0.3418 - accuracy: 0.9034 - val_loss: 0.1796 - val_accuracy: 0.9472
Epoch 2/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1434 - accuracy: 0.9578 - val_loss: 0.1221 - val_accuracy: 0.9620
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1017 - accuracy: 0.9695 - val_loss: 0.1034 - val_accuracy: 0.9671
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0778 - accuracy: 0.9763 - val_loss: 0.0898 - val_accuracy: 0.9724
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0618 - accuracy: 0.9811 - val_loss: 0.0830 - val_accuracy: 0.9747
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0515 - accuracy: 0.9837 - val_loss: 0.0881 - val_accuracy: 0.9719
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0423 - accuracy: 0.9868 - val_loss: 0.0783 - val_accuracy: 0.9769
Epoch 

In [ ]:
eval_and_save(model)

Test loss: 0.10631316155195236
Test accuracy: 0.9771999716758728
Model: "mnist-100x2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 784)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               78500     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 logit (Dense)               (None, 10)                1010      
                                                                 
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


simple cnn model

In [10]:
model = Sequential(name='mnist-simple-cnn')
model.add(Conv2D(4, (3, 3), name='conv_1', input_shape=(28, 28, 1), kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Conv2D(4, (2, 2), strides=(2, 2), name='conv_2', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Flatten())
model.add(Dense(20, activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(10, name='logit', kernel_initializer=GlorotUniform(seed=SEED)))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 15s 6ms/step - loss: 0.5126 - accuracy: 0.8446 - val_loss: 0.2563 - val_accuracy: 0.9264
Epoch 2/20
469/469 [==============================] - 2s 5ms/step - loss: 0.2473 - accuracy: 0.9281 - val_loss: 0.2291 - val_accuracy: 0.9355
Epoch 3/20
469/469 [==============================] - 2s 5ms/step - loss: 0.2193 - accuracy: 0.9368 - val_loss: 0.2021 - val_accuracy: 0.9438
Epoch 4/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1996 - accuracy: 0.9424 - val_loss: 0.1883 - val_accuracy: 0.9447
Epoch 5/20
469/469 [==============================] - 3s 6ms/step - loss: 0.1833 - accuracy: 0.9464 - val_loss: 0.1856 - val_accuracy: 0.9468
Epoch 6/20
469/469 [==============================] - 3s 7ms/step - loss: 0.1690 - accuracy: 0.9506 - val_loss: 0.1670 - val_accuracy: 0.9527
Epoch 7/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1569 - accuracy: 0.9543 - val_loss: 0.1627 - val_accuracy: 0.9532
Epoch

In [11]:
eval_and_save(model)

Test loss: 0.13413523137569427
Test accuracy: 0.9632999897003174
Model: "mnist-simple-cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 26, 26, 4)         40        
                                                                 
 conv_2 (Conv2D)             (None, 13, 13, 4)         68        
                                                                 
 flatten (Flatten)           (None, 676)               0         
                                                                 
 dense (Dense)               (None, 20)                13540     
                                                                 
 logit (Dense)               (None, 10)                210       
                                                                 
Total params: 13,858
Trainable params: 13,858
Non-trainable params: 0
_______________________________________________

cnn with max pooling and dropout

In [ ]:
model = Sequential(name='mnist-sota')
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(200, activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dropout(0.5, seed=SEED))
model.add(Dense(200, activation='relu', kernel_initializer=GlorotUniform(seed=SEED)))
model.add(Dense(10, name='logit', kernel_initializer=GlorotUniform(seed=SEED)))

model.compile(loss=tfr.keras.losses.SoftmaxLoss(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 8s 11ms/step - loss: 0.2427 - accuracy: 0.9237 - val_loss: 0.0467 - val_accuracy: 0.9851
Epoch 2/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0687 - accuracy: 0.9798 - val_loss: 0.0285 - val_accuracy: 0.9914
Epoch 3/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0472 - accuracy: 0.9855 - val_loss: 0.0316 - val_accuracy: 0.9907
Epoch 4/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0388 - accuracy: 0.9886 - val_loss: 0.0372 - val_accuracy: 0.9883
Epoch 5/20
469/469 [==============================] - 4s 9ms/step - loss: 0.0315 - accuracy: 0.9909 - val_loss: 0.0254 - val_accuracy: 0.9927
Epoch 6/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0274 - accuracy: 0.9916 - val_loss: 0.0195 - val_accuracy: 0.9944
Epoch 7/20
469/469 [==============================] - 3s 7ms/step - loss: 0.0244 - accuracy: 0.9928 - val_loss: 0.0178 - val_accuracy: 0.9950
Epoch

In [ ]:
eval_and_save(model)

Test loss: 0.03171161562204361
Test accuracy: 0.9937999844551086
Model: "mnist-sota"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 64)          36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 64)         0 